In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('finger-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))


rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]


ix = float(config.get('Configuration', 'ix'))
iy = float(config.get('Configuration', 'iy'))

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]


In [2]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))


ts=list(range(t0,tN,time_step))

def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

day=list(range(day_time(t0),day_time(tN)+1))[0]


2024-05-11 00:03:00 End date is 1715378580 day: 19854 2024-05-11 00:03:00
2024-05-10 23:35:00
2024-05-10 23:35:00 Init date is 1715376900 day: 19853 2024-05-10 23:35:00


In [4]:
S={}
for user in users:
    S[user]={}
    series=read_csv(os.path.join(data_folder+"/DAY_"+str(day),"GT.ROOM."+user+".tsv"), sep="\t",parse_dates=True, header=None, names=["tt0","time", "scene"]) 
 
    for index, row in series.iterrows():
        S[user][row["time"]]=row["scene"]
print(S)

{'f02e': {1715376900: 's5', 1715376901: 's5', 1715376902: 's5', 1715376903: 's5', 1715376904: 's5', 1715376905: 's5', 1715376906: 's5', 1715376907: 's5', 1715376908: 's5', 1715376909: 's5', 1715376910: 's5', 1715376911: 's5', 1715376912: 's5', 1715376913: 's5', 1715376914: 's5', 1715376915: 's5', 1715376916: 's5', 1715376917: 's5', 1715376918: 's5', 1715376919: 's5', 1715376920: 's5', 1715376921: 's5', 1715376922: 's5', 1715376923: 's5', 1715376924: 's5', 1715376925: 's5', 1715376926: 's5', 1715376927: 's5', 1715376928: 's5', 1715376929: 's5', 1715376930: 's5', 1715376931: 's5', 1715376932: 's5', 1715376933: 's5', 1715376934: 's5', 1715376935: 's5', 1715376936: 's5', 1715376937: 's5', 1715376938: 's5', 1715376939: 's5', 1715376940: 's5', 1715376941: 's5', 1715376942: 's5', 1715376943: 's5', 1715376944: 's5', 1715376945: 's5', 1715376946: 's5', 1715376947: 's5', 1715376948: 's5', 1715376949: 's5', 1715376950: 's5', 1715376951: 's5', 1715376952: 's5', 1715376953: 's5', 1715376954: 's5', 

In [5]:
def getFPImg(tt):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0  

def getSceneImg(scene):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP.mean."+str(scene)+".png"))/255.0

In [6]:


XY0={}
rssi0={}
Y0={}

T0={}
T1={}
S0={}



anchors_data={}
for user in users:
    XY0[user]=[]
    rssi0[user]=[]
    Y0[user]=[]
    S0[user]=[]
    T1[user]=[]
    T0[user]=[]    


    anchors_data[user]={}
    for anchor in anchors:
        print(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
        print(user,anchor)
        series=read_csv(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d","d2"])
        series=series.values
        anchors_data[user][anchor]=series


for it,tt in enumerate(ts):
    imgX=getFPImg(tt)
    print("imgX",imgX.shape)

    #img00=np.zeros((w,h))
    #scene=getScene(tt)
    #if(scene):
    #    img00=getSceneImg(scene)
    #print("img00",img00)
    #imgX=imgX-img00
    #imgX[imgX<0]=0
    for user in users:
        
        label=S[user][tt]

        imgY=getGTImg(tt,user)
        print("user",user,"imgY",imgY.shape, label)
        #if(label is None or label=="IDLE"):
        #    continue

        dist=[]
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][1]
            #rint(anchors_data[user][anchor][it][0], "vs",tt)

            #if(d!=-1):
            #    d=(d - rssi_min) / (rssi_max - rssi_min)

            #print(anchors_data[user][anchor][it][0],anchors_data[user][anchor][it][1],anchors_data[user][anchor][it][2])
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            #print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
            dist.append(d)


        
        #dist2=[]
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][2]
            #rint(anchors_data[user][anchor][it][0], "vs",tt)

            #if(d!=-1):
            #    d=(d - rssi_min) / (rssi_max - rssi_min)

            #print(anchors_data[user][anchor][it][0],anchors_data[user][anchor][it][1],anchors_data[user][anchor][it][2])
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            #print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
            dist.append(d)

        dist=np.array(dist)
        print("dist:",dist)

        rssi0[user].append(dist.reshape(len(anchors*2),1))
        XY0[user].append(imgX[:, :, np.newaxis])
        Y0[user].append(imgY[:, :, np.newaxis])
        T0[user].append(it)
        T1[user].append(tt)
        S0[user].append(label)
        


./data//DAY_19853/f02e.1fa0.tsv
f02e 1fa0
./data//DAY_19853/f02e.2eff.tsv
f02e 2eff
./data//DAY_19853/f02e.399a.tsv
f02e 399a
./data//DAY_19853/f02e.6aca.tsv
f02e 6aca
./data//DAY_19853/f02e.8171.tsv
f02e 8171
./data//DAY_19853/f02e.a304.tsv
f02e a304
./data//DAY_19853/f02e.c492.tsv
f02e c492
./data//DAY_19853/f02e.eb38.tsv
f02e eb38
./data//DAY_19853/3d57.1fa0.tsv
3d57 1fa0
./data//DAY_19853/3d57.2eff.tsv
3d57 2eff
./data//DAY_19853/3d57.399a.tsv
3d57 399a
./data//DAY_19853/3d57.6aca.tsv
3d57 6aca
./data//DAY_19853/3d57.8171.tsv
3d57 8171
./data//DAY_19853/3d57.a304.tsv
3d57 a304
./data//DAY_19853/3d57.c492.tsv
3d57 c492
./data//DAY_19853/3d57.eb38.tsv
3d57 eb38
./data//DAY_19853/ed9c.1fa0.tsv
ed9c 1fa0
./data//DAY_19853/ed9c.2eff.tsv
ed9c 2eff
./data//DAY_19853/ed9c.399a.tsv
ed9c 399a
./data//DAY_19853/ed9c.6aca.tsv
ed9c 6aca
./data//DAY_19853/ed9c.8171.tsv
ed9c 8171
./data//DAY_19853/ed9c.a304.tsv
ed9c a304
./data//DAY_19853/ed9c.c492.tsv
ed9c c492
./data//DAY_19853/ed9c.eb38.tsv
ed

imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.4342 0.     0.     0.4895 0.     0.     0.609  0.     0.4342
 0.     0.     0.4895 0.     0.     0.609 ]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.4001 0.     0.     0.4661 0.     0.     0.705  0.     0.3757
 0.     0.     0.4572 0.     0.     0.6971]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.3822 0.     0.     0.4816 0.     0.     0.6863 0.     0.3761
 0.     0.     0.4319 0.     0.     0.6699]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.3822 0.     0.     0.4886 0.     0.     0.6745 0.     0.3813
 0.     0.     0.4858 0.     0.     0.6745]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.3832 0.     0.     0.4839 0.     0.     0.6745 0.     0.3818
 0.     0.     0.483  0.     0.     0.6741]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.3869 0.     0.     0.5031 0.     0.     0.6731 0.   

imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.675  0.     0.2927 0.02   0.     0.     0.5809 0.     0.675
 0.     0.2927 0.02   0.     0.     0.5809]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0.     0.     0.     0.2927 0.0223 0.     0.     0.587  0.     0.
 0.     0.2927 0.0223 0.     0.     0.587 ]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s1
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18,

user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.3921 0.     0.     0.3471 0.     0.     0.7481 0.     0.3836
 0.     0.     0.3195 0.     0.     0.7415]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.453  0.     0.     0.3963 0.1569 0.     0.7336 0.     0.3977
 0.     0.     0.3954 0.1395 0.     0.6961]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.4703 0.     0.109  0.3897 0.0824 0.     0.7476 0.     0.4267
 0.     0.109  0.3832 0.0824 0.     0.727 ]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.3251 0.5055 0.     0.     0.3888 0.0875 0.     0.734  0.3251 0.4844
 0.     0.     0.3888 0.0528 0.     0.7265]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.3016 0.5045 0.     0.     0.386  0.073  0.     0.6985 0.2918 0.4975
 0.  

user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.6788 0.     0.3007 0.0749 0.     0.     0.5715 0.     0.6788
 0.     0.3007 0.0749 0.     0.     0.5715]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.675  0.     0.2979 0.0781 0.     0.     0.5744 0.     0.675
 0.     0.2979 0.0781 0.     0.     0.5744]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
di

...

imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.    0.43  0.    0.    0.    0.    0.    0.409 0.    0.43  0.    0.
 0.    0.    0.    0.409]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.     0.     0.     0.7621 0.3719 0.     0.3373 0.     0.
 0.     0.     0.6961 0.3719 0.     0.3373]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.6849 0.     0.     0.     0.6736 0.3668 0.     0.3664 0.6849 0.
 0.     0.     0.6736 0.3668 0.     0.3467]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.7251 0.2529 0.     0.     0.6582 0.     0.     0.3063 0.7251 0.2529
 0.     0.     0.6582 0.     0.     0.3063]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     

imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0725 0.     0.     0.8393 0.2098 0.     0.1943 0.     0.0725
 0.     0.     0.8393 0.2098 0.     0.1943]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0791 0.     0.     0.8351 0.2126 0.     0.1967 0.     0.0791
 0.     0.     0.8351 0.2126 0.     0.1967]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY

imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.     0.     0.     0.8356 0.2079 0.     0.1943 0.     0.
 0.     0.     0.8356 0.2079 0.     0.1943]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0805 0.     0.     0.8361 0.207  0.     0.1962 0.     0.0805
 0.     0.     0.8361 0.207  0.     0.1962]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18

user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0814 0.     0.     0.     0.207  0.     0.1948 0.     0.0814
 0.     0.     0.     0.207  0.     0.1948]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0819 0.     0.     0.837  0.2126 0.     0.199  0.     0.0819
 0.     0.     0.837  0.2126 0.     0.199 ]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
d

imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.073  0.     0.     0.8365 0.     0.     0.199  0.     0.073
 0.     0.     0.8365 0.     0.     0.199 ]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0.     0.0805 0.     0.     0.837  0.2094 0.     0.2    0.     0.0805
 0.     0.     0.837  0.2094 0.     0.2   ]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user ed9c imgY 

In [7]:
from collections import Counter

for user in users:
    XY0[user]=np.array(XY0[user])
    rssi0[user]=np.array(rssi0[user])
    Y0[user]=np.array(Y0[user])
    S0[user]=np.array(S0[user])
    print(Counter(S0[user]))
    T0[user]=np.array(T0[user])
    T1[user]=np.array(T1[user])
    minT=np.min(T0[user])
    maxT=np.max(T0[user])


    print(minT)
    print(maxT)

Counter({'s5': 1680})
0
1679
Counter({'s5': 1680})
0
1679
Counter({'s5': 1680})
0
1679


In [8]:
XYT=[]
rssiT=[]
YT=[]

window_size=5

T=window_size

TA=math.ceil(window_size/2)
TB=math.floor(window_size/2)


print(TA,TB)

SS=[]
for user in users:
    minT=np.min(T0[user])
    maxT=np.max(T0[user])
    for t0 in range(minT+TA,maxT-TB):        
        print(user,label,t0,T0[user][t0],T1[user][t0])

        
        tt=T1[user][t0]

        label=S0[user][t0]
        
        if(label=="IDLE"):
            continue
        #scene=getScene(tt)
        #print("Scene",scene,tt,t)
        #if not scene:
        #    continue

        #t=T0[user][t0]
        #subW=list(range(t-TA,t+TB))
        #print("subW:",subW, "t:",t,t0)
        #if not (set(subW).intersection(set(T0)) == set(subW)):
        #    print("Not data for sliding window!",t0)
        #    continue

        if(all(map(lambda x: x <= 0, rssi0[user][t0-TA:t0+TB].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t0)
            continue

        #if(all(map(lambda x: x <= 0, rssi0[user][t0:t0+TB].flatten()))):
        #    print("Distance -1 for RSSI in sliding window!",t0)
        #    continue

        if(all(map(lambda x: x == 0, Y0[user][t0].flatten()))):
            print("Distance 0 for Y0 in t!",t0)
            continue


        if(XY0[user][t0-TA:t0+TB].shape[0]!=T):
            print("#",XY0[user][t0-TA:t0+TB].shape)
            continue
        if(rssi0[user][t0-TA:t0+TB].shape[0]!=T):
            print("@",rssi0[user][t0-TA:t0+TB].shape)
            continue

        XYT.append(XY0[user][t0-TA:t0+TB])
        rssiT.append(rssi0[user][t0-TA:t0+TB])
        YT.append(Y0[user][t0])
        #print(T1[t0],T0[t0])
        SS.append(label)
XYT=np.array(XYT)
rssiT=np.array(rssiT)
YT=np.array(YT)    
SS=np.array(SS)    
print(XYT.shape)
print(rssiT.shape)
print(YT.shape)
print(SS.shape)


3 2
f02e s5 3 3 1715376903
f02e s5 4 4 1715376904
f02e s5 5 5 1715376905
f02e s5 6 6 1715376906
f02e s5 7 7 1715376907
f02e s5 8 8 1715376908
f02e s5 9 9 1715376909
f02e s5 10 10 1715376910
f02e s5 11 11 1715376911
f02e s5 12 12 1715376912
f02e s5 13 13 1715376913
f02e s5 14 14 1715376914
f02e s5 15 15 1715376915
f02e s5 16 16 1715376916
f02e s5 17 17 1715376917
f02e s5 18 18 1715376918
f02e s5 19 19 1715376919
f02e s5 20 20 1715376920
f02e s5 21 21 1715376921
f02e s5 22 22 1715376922
f02e s5 23 23 1715376923
f02e s5 24 24 1715376924
f02e s5 25 25 1715376925
f02e s5 26 26 1715376926
f02e s5 27 27 1715376927
f02e s5 28 28 1715376928
f02e s5 29 29 1715376929
f02e s5 30 30 1715376930
f02e s5 31 31 1715376931
f02e s5 32 32 1715376932
f02e s5 33 33 1715376933
f02e s5 34 34 1715376934
f02e s5 35 35 1715376935
f02e s5 36 36 1715376936
f02e s5 37 37 1715376937
f02e s5 38 38 1715376938
f02e s5 39 39 1715376939
f02e s5 40 40 1715376940
f02e s5 41 41 1715376941
f02e s5 42 42 1715376942
f02e s5 43

In [9]:
import matplotlib.pyplot as plt

L=20

# Create a figure with two subplots (one row, two columns)
fig, axes = plt.subplots(L, 2, figsize=(200, 200))


for k in range(0,L):
    mpAll=XY0[users[0]][k+100].reshape(w,h)

    axes[k,0].imshow(mpAll, cmap='viridis', interpolation='nearest')
    axes[k,0].set_title('All players (last frame)')
    axes[k,0].axis('off')

    mpOne=Y0[users[0]][k+100].reshape(w,h)

    axes[k,1].imshow(mpOne, cmap='viridis', interpolation='nearest')
    axes[k,1].set_title('True (last frame) of player')
    axes[k,1].axis('off')

    
# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [10]:
def contrastive_loss2(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted
	# class label data type (otherwise we run the risk of having two
	# separate data types, causing TensorFlow to error out)
	y = tf.cast(y, preds.dtype)
	# calculate the contrastive loss between the true labels and
	# the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
	# return the computed contrastive loss to the calling function
	return loss
from tensorflow.python.framework import ops

def contrastive_loss3(y_true, y_pred, margin=1.0):
    """Calculates the contrastive loss.

    Arguments:
        y_true: Tensor of labels (0 or 1), of type float32.
        y_pred: Tensor of predictions of the same length as y_true, each of type float32.
        margin: Margin value for the contrastive loss, default is 1.0.

    Returns:
        A tensor containing the contrastive loss as a floating-point value.
    """

    # Calculate squared distances
    square_pred = tf.math.square(y_pred)
    margin_square = tf.math.square(tf.math.maximum(margin - y_pred, 0))
    
    # Compute contrastive loss
    loss = tf.reduce_mean((1 - y_true) * square_pred + y_true * margin_square)

    return loss


2024-06-24 18:10:50.333756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 18:10:51.260716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
from sklearn.model_selection import train_test_split
from keras.models import load_model

train_indices=np.arange(XYT.shape[0])
np.random.shuffle(train_indices)
print("TRAIN:", train_indices)

XYT_train=XYT[train_indices]
rssiT_train=rssiT[train_indices]
YT_train=YT[train_indices]
    
autoencoder = load_model("autoencoder.model3.all.50.h5",custom_objects={'contrastive_loss3':                   
contrastive_loss3})


history = autoencoder.fit([XYT_train,rssiT_train] , YT_train,
                    batch_size=16,
            epochs=25)

autoencoder.save("autoencoder.model4.all.50.h5")

TRAIN: [2428  503 2846 ... 4720 4165 1794]


2024-06-24 18:10:53.264328: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 18:10:53.299290: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 18:10:53.299364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 18:10:53.301111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 18:10:53.301169: I tensorflow/compile

Epoch 1/25


2024-06-24 18:10:55.083425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-24 18:10:55.084972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-24 18:10:55.085920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

298/299 [============================>.] - ETA: 0s - loss: 0.0366 - mse: 3.2153e-04 - mae: 0.0019

2024-06-24 18:11:56.959276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


299/299 [==============================] - 63s 191ms/step - loss: 0.0366 - mse: 3.2146e-04 - mae: 0.0019
Epoch 2/25
299/299 [==============================] - 57s 189ms/step - loss: 0.0344 - mse: 9.9672e-06 - mae: 3.1610e-04
Epoch 3/25
299/299 [==============================] - 59s 197ms/step - loss: 0.0344 - mse: 6.6250e-06 - mae: 2.3380e-04
Epoch 4/25
299/299 [==============================] - 62s 206ms/step - loss: 0.0345 - mse: 1.9114e-05 - mae: 2.7209e-04
Epoch 5/25
299/299 [==============================] - 61s 204ms/step - loss: 0.0345 - mse: 2.1537e-05 - mae: 3.2847e-04
Epoch 6/25
299/299 [==============================] - 61s 203ms/step - loss: 0.0344 - mse: 5.7939e-06 - mae: 1.9116e-04
Epoch 7/25
299/299 [==============================] - 57s 192ms/step - loss: 0.0344 - mse: 5.5714e-06 - mae: 1.7087e-04
Epoch 8/25
299/299 [==============================] - 57s 192ms/step - loss: 0.0344 - mse: 7.6376e-06 - mae: 2.0723e-04
Epoch 9/25
299/299 [==============================] - 5